In [ ]:
# Пример вызова генплана на большой территории 

import geopandas as gpd
from shapely import Polygon, MultiPolygon

from app.gen_planner.python.src import GenPlanner, gen_plan,basic_func_zone

zones, roads = GenPlanner(gpd.read_file('1.gpkg')).poly2func2terr2block(gen_plan)

In [ ]:
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', categorical=True)
m1

In [ ]:
# тут вызовы по сценариям по разным территориям, везде basic по умолчанию, но другие лежат в zoning.


from app.gen_planner.python.src import GenPlanner,basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner,basic_func_zone
import geopandas as gpd
zones, roads = GenPlanner(gpd.read_file('Аэродром_лодейнопольское_поселение.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner,basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
# тут вызов деления территориальной зоны на кварталы по минимальному размеру терр.зоны
from app.gen_planner.python.src import GenPlanner, terr_zones
import geopandas as gpd
zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).poly2block(terr_zones.residential_terr)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [3]:
from app.gen_planner.python.src import GenPlanner
import geopandas as gpd
zones = {'1':0.1,
         '2':0.1,
         '3':0.1,
         '4':0.1,
         '5':0.1,
         '6':0.1,
         '7':0.1,
         '8':0.1,}
zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).split_poly(zones,roads_width=50)
zones.explore()

In [4]:
import geopandas as gpd
import numpy as np 
from  shapely import MultiPolygon,Polygon
local_crs = 32636
multip = gpd.read_file('2.gpkg').to_crs(local_crs)

def elasatic_wrap(gdf: gpd.GeoDataFrame) -> Polygon:
    gdf = gdf.copy()
    multip = gpd.GeoDataFrame(geometry=gdf.geometry,crs = gdf.crs).explode(ignore_index=True)
    max_dist =  np.ceil(multip.apply(lambda row: multip.drop(row.name).distance(row.geometry).min(), axis=1).max(axis=0))
    poly = multip.buffer(max_dist).union_all().buffer(-max_dist)
    if isinstance(poly, MultiPolygon):
        raise RuntimeError('Cant cast Multipolygon to Polygon')
    poly = Polygon(poly.exterior)
    return poly
p = elasatic_wrap(multip)

In [6]:
gpd.GeoDataFrame(geometry=[p.simplify(250)],crs=local_crs).explore()

In [7]:
from app.gen_planner.python.src import GenPlanner,gen_plan
import geopandas as gpd

gp = GenPlanner(gpd.read_file('2.gpkg'),rotation=0)
multiterritory = gp.transformed_poly
zones,roads = gp.poly2func2terr2block(gen_plan)

In [8]:
zones.explore()

In [ ]:
m1 = zones.explore(column='func_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', categorical=True)
m1

In [ ]:
d = gpd.read_file('project Шлиссельбург.geojson')
d.explore(tiles='CartoDB positron')

In [ ]:

from app.gen_planner.python.src import GenPlanner,basic_func_zone,func_zones
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).poly2terr(func_zones.residential_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)